In [ ]:
%%time
!pip install pycocoevalcap
!pip install bert_score

In [ ]:
## 计算模型测试结果的bleu4和bert_score
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from bert_score import BERTScorer
import numpy as np

bleu4_scorers = {"Bleu": Bleu(4)}
bert_scorer = BERTScorer(lang="en",
                         model_type='roberta-large',
                         device='cuda:1',
                         rescale_with_baseline=True
                        )

# 测评问题的流畅性
def fluencyScore(preds_list, gold_list):
    gts = {}
    res = {}
    for i, (p, g) in enumerate(zip(preds_list, gold_list)):
        gts[i] = [p]
        res[i] = [g]
    scores = {}
    for name, scorer in bleu4_scorers.items():
        score, all_scores = scorer.compute_score(gts, res)
        if isinstance(score, list):
            for i, sc in enumerate(score, 1):
                scores[name + str(i)] = sc
        else:
            scores[name] = score
    return scores,all_scores[-1]

# 测评语义相似度
def SemanticScore(preds_list, gold_list):
    p,r,f1 = bert_scorer.score(preds_list, gold_list, verbose=True)
    bert_score = np.mean(f1.tolist())
    return bert_score

## 合并结果
def getTotalScore(preds_list,gold_list):
    bert_score = SemanticScore(preds_list,gold_list)
    scores,all_scores = fluencyScore(preds_list,gold_list)
    last_score = (bert_score/2+scores['Bleu4']/2)*100
    return {'TotalScore':last_score, 
            'BERTScore':bert_score,
            'Bleu1':scores['Bleu1'],
            'Bleu2':scores['Bleu2'],
            'Bleu3':scores['Bleu3'],
            'Bleu4':scores['Bleu4'],
           },all_scores


In [4]:
"""
model_name: 模型名称，本代码中支持不同T5型号
train_data_path,dev_data_path,test_data_path:三个数据集路径
"""
class Args:
    model_name = 'google/flan-t5-base'
    train_data_path = '/kaggle/input/scnu-ai-challenge-5/train.json'
    dev_data_path = '/kaggle/input/scnu-ai-challenge-dataset-with-sorted-pred-facts/dev.json'
    test_data_path = '/kaggle/input/scnu-ai-challenge-dataset-with-sorted-pred-facts/test.json'

## 用于记录训练过程中的loss和score
class MyLogging:
    best_score = 0
    loss_list=[]
    scores_list=[]

args = Args()
logging=MyLogging()

def plot_loss():
    import matplotlib.pyplot as plt
    loss_list=logging.loss_list
    plt.plot(range(len(loss_list)),loss_list)
    plt.show()

In [5]:
import json
import random
from tqdm import tqdm
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

import torch
from torch import nn
import torch.nn.functional as F

## 数据整理器
from transformers import DataCollatorForSeq2Seq
## 模型及分词器
from transformers import T5ForConditionalGeneration as ConditionalGeneration
from transformers import AutoTokenizer
## 优化器
from transformers.optimization import Adafactor, AdafactorSchedule


2024-06-25 12:54:48.488604: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 12:54:48.488697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 12:54:48.619911: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
class Trainer_config:
    pg_num=1
    max_source_length=1024
    max_target_length=128
    training_batch=512
    testing_batch=10
    updata_batch=8
    epochs=2
    batch_size=16
    eval_model=True
def get_trainer_config(pg_num,max_source_length,batch_size,updata_batch,epochs):
    config=Trainer_config()
    config.pg_num=pg_num
    config.max_source_length=max_source_length
    config.batch_size=batch_size
    config.updata_batch=updata_batch
    config.epochs=epochs
    return config

## 测试
def get_eval_config():
    config=Trainer_config()
    config.pg_num=7
    config.max_source_length=1024
    config.batch_size=4
    return config
def show_config(config):
    print('pg_num:',config.pg_num)
    print('max_souce_length:',config.max_source_length)
    print('train_batch:',config.training_batch)
    print('updata_batch:',config.updata_batch)
    print('epochs:',config.epochs)
    print('batch_size:',config.batch_size)



In [7]:
## 读入数据
data_dict={}
def read_json(data_path,data_type):
    if data_dict.get(data_type,-1)!=-1:
        return data_dict[data_type]
    else:
        with open(data_path, 'r', encoding='utf-8') as f:
            data =json.load(f)
        data_dict[data_type]=data
        return data

## 清洗数据及准备数据   
def get_datas(data_path,data_type):
    data=read_json(data_path,data_type)
    answers=[]
    questions=[]
    title_to_contexts=[]
    support_facts=[]
    for i,d in tqdm(enumerate(data)):
        ## 删除训练集中的单跳问题
        if data_type=='train':
            if d['level']=='easy':
                continue
        
        ## 验证集只保留难度为hard的文段，结果较为稳定
        if data_type=='dev':
            if d['level']!='hard':
                continue
        
        answer=d['answer']
        
        answer_token_num=len(answer.split(' '))
        ## 根据1-0.25*len的概率删除数据
        if data_type=='train':
            p_drop=max(1-0.25*answer_token_num,0)
            if random.random()<p_drop:
                continue
                
        ## support_facts
        support_facts_list=[]
        fact_set=set()
        if data_type=='train':
            titles=[]
            for item in d['supporting_facts']:
                fact_set.add(item[0])
            support_facts_list=list(fact_set)
        else:
            for item in d['pred_support_facts']:
                support_facts_list.append(item[0])
     
    
        ## title_to_context
        title_to_context={}   
        for x in d['context']:
            s=' '.join(x[1])
            context=f'{x[0]}. {s} '
            title_to_context[x[0]]=context
        
        ## question
        question=[]
        if data_type!='test':
            question=d['question']
        
        ## append
        questions.append(question)
        title_to_contexts.append(title_to_context)
        answers.append(answer)
        support_facts.append(support_facts_list)
        
    return title_to_contexts,questions,answers,support_facts

In [8]:
%%time
## 生成训练集并打乱
train_T2C,train_Q,train_A,train_SF=get_datas(args.train_data_path,'train')
def get_infinite_data(datas):
    while True:
        for value in datas:
            yield value

train_data=list(zip(train_T2C,train_Q,train_A,train_SF))
print('train_data_num:',len(train_data))
random.shuffle(train_data)
train_data_generator=get_infinite_data(train_data)
def get_example():
    return train_data_generator.__next__()

## 将数据整理成输入格式
def get_context(title_to_context,support_facts_list,data_type='train',pg_num=7):
    if data_type=='train':
        if len(support_facts_list)>pg_num:
                support_facts_list=support_facts_list[:pg_num]
        else:
            num=pg_num-len(support_facts_list)
            titles=[]
            for title in list(title_to_context.keys()):
                if title not in support_facts_list:
                    titles.append(title)
            support_facts_list.extend(random.choices(titles,k=min(len(titles),num)))
        random.shuffle(support_facts_list)
    else:
        support_facts_list=support_facts_list[:pg_num]
        
    context=''   
    for title in support_facts_list:
        context+=title_to_context[title]
    return context

title_to_context,question,answer,support_facts_list=get_example()
get_context(title_to_context,support_facts_list,data_type='train',pg_num=2)

88947it [00:00, 118844.51it/s]

train_data_num: 37382
CPU times: user 7.99 s, sys: 2.27 s, total: 10.3 s
Wall time: 19.3 s


'Albany Medical College. Albany Medical College (AMC) is a medical school located in Albany, New York, United States.  It was founded in 1839 by Alden March and James H. Armsby and is one of the oldest medical schools in the nation.  The college is part of the Albany Medical Center, which includes the Albany Medical Center Hospital. Henry Hun. Henry Hun (March 21, 1854 – March 14, 1924) an American physician, was professor of Nervous Diseases at the Albany Medical College in New York for 30 years.  He published several unique teaching volumes for his students as well as numerous journal articles on neurological disorders. '

In [17]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self,
                 title_to_contexts,
                 support_facts_lists,
                 questions,
                 answers,
                 tokenizer,
                 config,
                 data_type):
        self.title_to_contexts=title_to_contexts
        self.support_facts_lists=support_facts_lists
        self.questions = questions
        self.answers=answers
        self.tokenizer = tokenizer
        self.config=config
        self.data_type=data_type

    def __len__(self):
        if self.data_type=='train':
            return self.config.epochs*self.config.training_batch*self.config.batch_size
        return len(self.answers)
    
    def __getitem__(self,item):
        if self.data_type=='train':
            title_to_context,question,answer,support_facts_list=get_example()
        else:
            title_to_context= self.title_to_contexts[item]
            support_facts_list=self.support_facts_lists[item]
            answer=self.answers[item]
            question=''
            if self.data_type=='dev':
                question=self.questions[item]

        context=get_context(title_to_context,
                            support_facts_list,
                            self.data_type,
                            self.config.pg_num)

        source=f'Please generate only one question according to the answer and the paragraphs.\n\
                   Answer: {answer} \n Content: {context}'
        target=question
            
        source_encoding = self.tokenizer(text=source,
                      max_length=self.config.max_source_length,
                      padding=True,
                      truncation=True,
                      add_special_tokens=True,
                      return_attention_mask=True,
                      )
        if self.data_type=='test':
            return {
                "input_ids":source_encoding['input_ids'],
               "attention_mask":source_encoding['attention_mask'],
               }
        
        target_encoding=self.tokenizer(text=target,
                      max_length=self.config.max_target_length,
                      padding=True,
                      truncation=True,
                      add_special_tokens=True,
                      return_attention_mask=True,
                    )
   

        return {
                "input_ids":torch.LongTensor(source_encoding['input_ids']),
                "attention_mask":torch.LongTensor(source_encoding['attention_mask']),
                "labels":torch.LongTensor(target_encoding['input_ids']),
               }

    
def create_data_loader(data_path,data_type,tokenizer,config,shuffle=True):
    pg_num=config.pg_num
    max_len=config.max_source_length
    if data_type=='train':
        batch_size=config.batch_size
        title_to_contexts=None
        questions=None
        answers=None
        support_facts=None
    else:
        title_to_contexts,questions,answers,support_facts=get_datas(data_path,data_type)
        batch_size=config.batch_size*2

    ds = MyDataSet(
                   title_to_contexts=title_to_contexts,
                   support_facts_lists=support_facts,
                   questions=questions,
                   answers=answers,
                   tokenizer = tokenizer,
                   config=config,
                   data_type=data_type
                  )
    
    collate_fn = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8,padding=True)
    
    return torch.utils.data.DataLoader(ds,batch_size=batch_size,collate_fn=collate_fn,shuffle=shuffle)



In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
def load_model(model_name):
    print('model_name:',model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)
    model = ConditionalGeneration.from_pretrained(model_name,device_map="auto")#.to(device)
    print('Successfully loaded model')
    return model,tokenizer
def load_checkpoint(checkpoint):
    if checkpoint:
        try:
            model.load_state_dict(torch.load(checkpoint))
            print('Successfully loaded checkpoint!')
        except:
            print('Failed to load checkpoint!')
    


## 加载模型和检查点
model_name='t5-base'
checkpoint='/kaggle/input/t5-base-checkpoint5/checkpoint_best (34.90).pkl'
if model_name==None:
    model_name=args.defalut_model_name
model,tokenizer=load_model(model_name)
load_checkpoint(checkpoint)

optimizer = Adafactor(model.parameters(), relative_step=True, warmup_init=True,lr=None,clip_threshold=1.0)
scheduler = AdafactorSchedule(optimizer)

model_name: t5-base


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Successfully loaded model
Failed to load checkpoint!


In [11]:
%%time
## about 23s
def prepare_data(tokenizer,config):
    train_data_path=args.train_data_path
    dev_data_path=args.dev_data_path
    test_data_path=args.test_data_path
    print('preparing data...')
    train_data_loader=create_data_loader(train_data_path,'train',tokenizer,config,shuffle=True,)
    dev_data_loader=create_data_loader(dev_data_path,'dev',tokenizer,config,shuffle=False)
    test_data_loader=create_data_loader(test_data_path,'test',tokenizer,config,shuffle=False)
    print('prepare data success!')
    return train_data_loader,dev_data_loader,test_data_loader
train_data_loader,dev_data_loader,test_data_loader=prepare_data(tokenizer,config=get_eval_config())


preparing data...


1500it [00:00, 200735.63it/s]
7405it [00:00, 59629.88it/s]

prepare data success!
CPU times: user 1.03 s, sys: 593 ms, total: 1.62 s
Wall time: 2.93 s


In [ ]:
train_generator=get_infinite_data(train_data_loader)
def get_batch():
    return train_generator.__next__()
def get_answer(batch):
    bad_answers=[]
    for i in range(batch['input_ids'].shape[0]):
        bad_answers.append(tokenizer.convert_ids_to_tokens(batch['input_ids'][i])[10:20])
    return bad_answers
x=get_batch()
inputs=tokenizer.convert_ids_to_tokens(x['input_ids'][0])
#labels=tokenizer.convert_ids_to_tokens(x['labels'][0].reshape(-1))
x['input_ids'].shape,inputs

In [ ]:
## 评估模型函数
def eval_model(epoch=1,testing_batch=-1):
    model.eval()
    preds_list = []
    real_list=[]
    if testing_batch<=0:
        testing_batch=len(dev_data_loader)
    with tqdm(total=testing_batch, desc=f'Validation Epoch {epoch}', unit='batch') as pbar:
        for i,batch in enumerate(dev_data_loader):
            if i>testing_batch:break
            with torch.no_grad():
                input_ids = batch['input_ids'].to(device)
                attention_mask=batch['attention_mask'].to(device)
                labels=batch['labels'].to(device)
                if hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()
                generated_ids = model.generate(
                    input_ids=input_ids, 
                    attention_mask=attention_mask,
                    do_sample=False,
                    max_new_tokens=128,
                    num_beams=3, 
                )
                preds_list.extend(tokenizer.batch_decode(generated_ids, skip_special_tokens=True))
                labels=torch.where(labels!=-100,labels,tokenizer.pad_token_id)
                real_list.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))
                
            pbar.update(1)
    
    scores,all_score = getTotalScore(preds_list, real_list)
    for i in range(20):
        print('pred: ',preds_list[i])
        print("true: ",real_list[i])
        print('bleu4:',all_score[i])

    if scores['TotalScore'] > logging.best_score:
        torch.save(model.state_dict(),  "/kaggle/working/checkpoint_best.pkl")
        print(f"Total score: {logging.best_score} -> {scores['TotalScore'] }")
        print(f"checkpoint_best.pkl已存储至/kaggle/working/")
        logging.best_score = scores['TotalScore'] 

    logging.scores_list.append(scores)
    print(f"Scores: {scores}")
    return preds_list
pred_list=eval_model(epoch=0,testing_batch=10)

In [14]:
def train(model,config,show=False):
    if show:
        show_config(config)
        
    pg_num=config.pg_num
    training_batch=config.training_batch
    testing_batch=config.testing_batch
    updata_batch=config.updata_batch
    epochs=config.epochs
    global dev_data_loader
    train_data_loader,dev_data_loader,_=prepare_data(tokenizer,config)
    train_generator=get_infinite_data(train_data_loader)
    def get_batch():
        return train_generator.__next__()
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        batch_loss=0
        with tqdm(total=training_batch//updata_batch, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch',mininterval=3) as pbar:
            for i in range(training_batch):
                batch=get_batch()
                input_ids= batch['input_ids'].to(device)
                attention_mask=batch['attention_mask'].to(device)
                labels=batch['labels'].to(device)
 
                if hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()

                outputs = model(
                                input_ids=input_ids,
                                attention_mask=attention_mask,
                                labels=labels, 
                                return_dict=True,
                               )
                
               
                loss = outputs.loss
                loss.backward() 

                loss_t = loss.detach().cpu()
                epoch_loss += loss_t[0] if len(loss_t.shape) > 0 else loss_t.numpy()
                batch_loss +=loss_t[0] if len(loss_t.shape) > 0 else loss_t.numpy()
                
                if (i+1)%updata_batch==0:
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    batch_loss=batch_loss/updata_batch
                    #lr=optimizer.param_groups[0]['lr']
                    logging.loss_list.append(batch_loss)
                    pbar.set_postfix({
                        'loss': f'{ batch_loss:.4f}',
                        #'lr':f'{lr:.6f}'
                    })
                    batch_loss=0
                    pbar.update(1)
                    
        print(f"Average Loss: {epoch_loss / training_batch:.4f}")
        # 验证部分
        if config.eval_model:
            eval_model(epoch,testing_batch)
        torch.save(model.state_dict(), "/kaggle/working/checkpoint_last.pkl")

In [14]:
## stage1
config1={
    'pg_num':2,
    'max_source_length':384,
    'batch_size':12,
    'updata_batch':2,
    'epochs':10,
}
trainer_config=get_trainer_config(**config1)
train(model,trainer_config,True)

pg_num: 2
max_souce_length: 384
train_batch: 512
updata_batch: 2
epochs: 10
batch_size: 12
preparing data...


1500it [00:00, 137913.06it/s]
7405it [00:00, 56261.29it/s]


prepare data success!


Epoch 1/10: 100%|██████████| 256/256 [11:04<00:00,  2.60s/batch, loss=1.6687]


Average Loss: 2.0888


...

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 0.82 seconds, 107.69 sentences/sec
{'testlen': 1329, 'reflen': 1391, 'guess': [1329, 1241, 1153, 1065], 'correct': [428, 148, 62, 25]}
ratio: 0.9554277498195863
pred:  Are both plantago and plantago asiatica plants?
true:  Are Trichosanthes and Plantago both forms of plant life?
bleu4: 9.980099402512292e-13
pred:  What season of Indian reality TV series "Bigg Boss" did Lopamudra Raut contest?
true:  In which season of the Indian reality TV show "Big Boss" did the model Lopamundra Raut participate?
bleu4: 2.735838424446947e-05
pred:  Which senior United States Senator from Nevada was elected to his first full term in the United States Senate election in Wyoming, 2012?
true:  The 2012 United States Senate election in Nevada concluded with a close victory for which current Republican incumbent?
bleu4: 0.15939331383660452
pred:  When was the episode of "The Simpsons" that the character who manages Moe Goes from Rags to Riches first appeared?
true:  When was the Simpson's episode br

In [15]:
## stage2
config2={
    'pg_num':7,
    'max_source_length':1024,
    'batch_size':4,
    'updata_batch':8,
    'epochs':20,
}
trainer_config=get_trainer_config(**config2)
train(model,trainer_config,True)

pg_num: 7
max_souce_length: 1024
train_batch: 512
updata_batch: 8
epochs: 20
batch_size: 4
preparing data...


1500it [00:00, 157046.90it/s]
7405it [00:00, 61402.31it/s]


prepare data success!


Epoch 1/20: 100%|██████████| 64/64 [11:57<00:00, 11.22s/batch, loss=1.0425]


Average Loss: 1.0679


...

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.49 seconds, 89.06 sentences/sec
{'testlen': 635, 'reflen': 790, 'guess': [635, 591, 547, 503], 'correct': [291, 143, 82, 46]}
ratio: 0.8037974683534129
pred:  Are Plantago and Trichosanthes both types of plants?
true:  Are Trichosanthes and Plantago both forms of plant life?
bleu4: 1.1868405217902023e-12
pred:  Lopamudra Raut was a contestant of what season of Indian reality TV series "Bigg Boss"?
true:  In which season of the Indian reality TV show "Big Boss" did the model Lopamundra Raut participate?
bleu4: 2.3693055762206496e-05
pred:  Which Republican U.S. Senator was elected to the seat that was held on November 6, 2012 and was narrowly elected to his first full term over Shelley Berkley?
true:  The 2012 United States Senate election in Nevada concluded with a close victory for which current Republican incumbent?
bleu4: 2.3307710740126315e-13
pred:  Moe Szyslak first appeared in an episode that originally aired in the United States on what date?
true:  When was the Simps

Epoch 7/20:   0%|          | 0/64 [00:04<?, ?batch/s]


KeyboardInterrupt: 

## 在测试集上测试

In [ ]:
with open(args.test_data_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
test_id_list = [item['_id'] for item in data]

train_data_loader,dev_data_loader,test_data_loader=prepare_data(tokenizer,config=get_eval_config())
test_generator=get_infinite_data(test_data_loader)
def get_test_batch():
    return test_generator.__next__()

# 测试
generated_questions = []
generated_questions_dict = []
model.eval()
with tqdm(total=len(test_data_loader), desc=f'Test epoch {1}/{1}', unit='batch') as pbar:
    for i in range(len(test_data_loader)):
        batch=get_test_batch()
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask=batch['attention_mask'].to(device)
            generated_ids = model.generate(input_ids=input_ids, 
                                           max_new_tokens=128,
                                           do_sample=False)
            #preds_list.extend(tokenizer.batch_decode(generated_ids, skip_special_tokens=True))
        #generated_ids = generate_question(batch)
        qs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        generated_questions.extend(qs)
        pbar.update(1)

for _,item in enumerate(generated_questions):
    generated_questions_dict.append({'_id':test_id_list[_],'question':item})

with open('output.json', 'w', encoding='utf-8') as json_file:
        json.dump(generated_questions_dict, json_file, ensure_ascii=False, indent=4)